In [213]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import base64
import more_itertools as mit
import os

Challenge 9

PKCS#7, pad message to equal to multiple of block size, the pad value will be the pad length.

To unpad, simply read the pad value and chop off the pad.


In [235]:
def pkcs_7_pad(b: bytearray, blocksize):
  padding_length = blocksize - (len(b) % blocksize)
  return b+bytearray([padding_length]*padding_length)

def pkcs_7_unpad(b: bytearray):
  return b[:-b[-1]]

In [237]:
v = pkcs_7_pad(b"YELLOW SUBMARINE", 20)
o = pkcs_7_unpad(v)
v, o

(b'YELLOW SUBMARINE\x04\x04\x04\x04', b'YELLOW SUBMARINE')

Challenge 10

In [238]:
def aes_encrypt_128_block(key:bytearray, plaintext:bytearray):
  backend = default_backend()
  cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
  encryptor = cipher.encryptor()
  return encryptor.update(plaintext) + encryptor.finalize()

def aes_decrypt_128_block(key:bytearray, ciphertext:bytearray):
  backend = default_backend()
  cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
  decryptor = cipher.decryptor()
  return decryptor.update(ciphertext) + decryptor.finalize()

In [239]:
with open("10.txt", "r") as f:
  cipher_text = base64.b64decode(f.read())

In [243]:
def bxor(x:bytearray, y:bytearray):
  return bytearray([i^j for i,j in zip(x,y)])

def aes_encrypt_cbc(plaintext:bytearray, key:bytearray, iv:bytearray):
  prev_cipher_block = iv
  ciphertext = bytearray()
  for block in mit.chunked(pkcs_7_pad(plaintext, 16), 16):
    prev_cipher_block = aes_encrypt_128_block(key, bxor(block, prev_cipher_block))
    ciphertext += prev_cipher_block
  return ciphertext

def aes_decrypt_cbc(ciphertext:bytearray, key:bytearray, iv:bytearray):
  prev_cipher_block = iv
  plaintext = bytearray()
  for block in mit.chunked(ciphertext, 16):
    plaintext += bxor(prev_cipher_block, aes_decrypt_128_block(key, bytearray(block)))
    prev_cipher_block = block
  return pkcs_7_unpad(plaintext)

In [244]:
plain = bytearray(os.urandom(150))
key = os.urandom(16)
iv = os.urandom(16)
c = aes_encrypt_cbc(plain, key, iv)
o = aes_decrypt_cbc(c, key, iv)
o == plain

True

Challenge 11

In [255]:
import random

def aes_encrypt_ecb(plaintext:bytearray, key:bytearray):
  return aes_encrypt_128_block(key, pkcs_7_pad(plaintext, 16))

def encryption_oracle(plaintext:bytearray, key:bytearray):
  header, footer = os.urandom(random.randint(5, 10)), os.urandom(random.randint(5, 10))
  plaintext_pad = header+plaintext+footer

  # If True, CBC, else, ECB
  if random.choice([True, False]):
    iv = os.urandom(16)
    return aes_encrypt_cbc(plaintext_pad, key, iv), "CBC"
  return bytearray(aes_encrypt_ecb(plaintext_pad, key)), "ECB"

In [256]:
def detect_ecb(ct: bytearray):
  repeated = set(tuple(x) for x in list(mit.chunked(ct, 16)))
  if len(repeated) != len(ct)//16: return True
  return False

for _ in range(10):
  msg = bytearray(b"A"*50)
  key = os.urandom(16)
  v, mode = encryption_oracle(msg, key)
  pred_mode = "ECB" if detect_ecb(v) else "CBC"
  assert mode == pred_mode
  print(mode, pred_mode, mode == pred_mode)

CBC CBC True
CBC CBC True
CBC CBC True
CBC CBC True
ECB ECB True
ECB ECB True
CBC CBC True
ECB ECB True
ECB ECB True
ECB ECB True


Challenge 12

In [257]:
unknown_string = b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'

In [270]:
# Global secret key
secret_key = os.urandom(16)

# Find the block size
last_ctxt = None
for i in range(10):
  msg = bytearray(b"A"*(i+1)) + base64.b64decode(unknown_string)
  c = aes_encrypt_ecb(msg, secret_key)
  if last_ctxt is not None and (blocksize:=len(c) - len(last_ctxt)) != 0:
    print(len(msg), len(v), blocksize)
  last_ctxt = c

144 160 16


In [325]:
known_plaintext = bytearray()
for i in range(16):
  your_string = b"A"*(15-i) + known_plaintext
  print(your_string, len(your_string))
  target_block = aes_encrypt_ecb(your_string + base64.b64decode(unknown_string), secret_key)[:8]
  print(target_block)
  for b in range(2**8):
    header = your_string + bytes([b])
    msg =  header + base64.b64decode(unknown_string) 
    # print(header)
    c = aes_encrypt_ecb(msg, secret_key)
    if c[:8] == target_block:
      print(header, c[:8], target_block)
      known_plaintext.append(b)
# print(known_plaintext)

b'AAAAAAAAAAAAAAA' 15
b'\x9cZ\x7f\xe9\xed$\xa2M'
b'AAAAAAAAAAAAAAAR' b'\x9cZ\x7f\xe9\xed$\xa2M' b'\x9cZ\x7f\xe9\xed$\xa2M'
b'AAAAAAAAAAAAAAR' 15
b'\x9f\x11\x02\xe1k$\xb5d'
b'AAAAAAAAAAAAAARR' b'\x9f\x11\x02\xe1k$\xb5d' b'\x9f\x11\x02\xe1k$\xb5d'
b'AAAAAAAAAAAAARR' 15
b'\xacw\x9e\xe2\xd2\xe54G'
b'AAAAAAAAAAAAARRR' b'\xacw\x9e\xe2\xd2\xe54G' b'\xacw\x9e\xe2\xd2\xe54G'
b'AAAAAAAAAAAARRR' 15
b'\xc5:\x8clg\xcdKR'
b'AAAAAAAAAAAARRRR' b'\xc5:\x8clg\xcdKR' b'\xc5:\x8clg\xcdKR'
b'AAAAAAAAAAARRRR' 15
b'Eh\xfa\xe7\xdd"\xb3X'
b'AAAAAAAAAAARRRRR' b'Eh\xfa\xe7\xdd"\xb3X' b'Eh\xfa\xe7\xdd"\xb3X'
b'AAAAAAAAAARRRRR' 15
b'\x95Gf\xd3Bx\xfaO'
b'AAAAAAAAAARRRRRR' b'\x95Gf\xd3Bx\xfaO' b'\x95Gf\xd3Bx\xfaO'
b'AAAAAAAAARRRRRR' 15
b'\xc1\x7f\xc3I\xaa\xb6*\x93'
b'AAAAAAAAARRRRRRR' b'\xc1\x7f\xc3I\xaa\xb6*\x93' b'\xc1\x7f\xc3I\xaa\xb6*\x93'
b'AAAAAAAARRRRRRR' 15
b'\x9b]\xd5\x82"\xda`X'
b'AAAAAAAARRRRRRRR' b'\x9b]\xd5\x82"\xda`X' b'\x9b]\xd5\x82"\xda`X'
b'AAAAAAARRRRRRRR' 15
b'\x14G\x10\xff\xfdD\xec\xa8'
b'AAAAAAA

In [320]:
aes_encrypt_ecb(base64.b64decode(unknown_string), secret_key)[:8]

b'}+B\x9d\xffg*\xc0'

In [321]:
aes_encrypt_ecb(b'R'*16, secret_key)[:8]

b"\xa3\xd7'\x037\xd0\xa1\xa5"